# BWM Paper Figure 1 coverage

This code depends on the bwm trajectories, which are pre-computed in bwm_all_traj.csv (or can be re-computed using the relevant scripts)

In [19]:
import oursin as urchin
import pandas as pd
import numpy as np
from iblatlas.atlas import Insertion, AllenAtlas
from one.api import ONE
from brainwidemap import bwm_query

In [20]:
def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb


def vec_dist(coord0, coord1):
    return np.sqrt(np.sum(np.power([coord0[0]-coord1[0],coord0[1]-coord1[1],coord0[2]-coord1[2]],2)))


In [21]:

ba = AllenAtlas(25)
one = ONE()
# one = ONE(base_url='https://openalyx.internationalbrainlab.org',
#           password='international', silent=True)
df = bwm_query(one)
len(df)

Loading bwm_query results from fixtures/2023_12_bwm_release.csv


699

In [22]:
df

,pid,eid,probe_name,session_number,date,subject,lab
0,56f2a378-78d2-4132-b3c8-8c1ba82be598,6713a4a7-faed-4df2-acab-ee4e63326f8d,probe00,1,2020-02-18,NYU-11,angelakilab
1,47be9ae4-290f-46ab-b047-952bc3a1a509,56956777-dca5-468c-87cb-78150432cc57,probe01,1,2020-02-21,NYU-11,angelakilab
2,6be21156-33b0-4f70-9a0f-65b3e3cd6d4a,56956777-dca5-468c-87cb-78150432cc57,probe00,1,2020-02-21,NYU-11,angelakilab
3,8dfb86c8-d45c-46c4-90ec-33078014d434,4364a246-f8d7-4ce7-ba23-a098104b96e4,probe01,1,2020-01-20,NYU-12,angelakilab
4,c893c0a3-5597-49cf-baa1-60efdfdef542,b182b754-3c3e-4942-8144-6ee790926b58,probe01,1,2020-01-21,NYU-12,angelakilab
...,...,...,...,...,...,...,...
694,8bf0f1a4-0d8c-4df3-a99e-f7c81c809652,993c7024-0abc-4028-ad30-d397ad55b084,probe01,1,2020-09-16,CSH_ZAD_029,zadorlab
695,5d570bf6-a4c6-4bf1-a14b-2c878c84ef0e,fece187f-b47f-4870-a1d6-619afe942a7d,probe01,1,2020-09-17,CSH_ZAD_029,zadorlab
696,f7c93877-ec05-4091-a003-e69fae0f2fa8,fece187f-b47f-4870-a1d6-619afe942a7d,probe00,1,2020-09-17,CSH_ZAD_029,zadorlab
697,675952a4-e8b3-4e82-a179-cc970d5a8b01,c7bd79c9-c47e-4ea5-aea3-74dda991b48e,probe01,1,2020-09-19,CSH_ZAD_029,zadorlab


In [26]:
# Create bw dataframe containing columns on insertion position
COLUMN_NAMES = ['pid', 'phi_e', 'theta_e',
                'ml_t_e', 'ap_t_e', 'dv_t_e',
                'ml_e_e', 'ap_e_e', 'dv_e_e']
bwm = pd.DataFrame(columns=COLUMN_NAMES)
bwm['pid'] = df['pid']

# from needles2.probe_model import ProbeModel
# pr = ProbeModel(ba=ba, lazy=True)

for i,row in df.iterrows():
    # Get trajectory (TODO assumed here all have histology track)
    traj = one.alyx.rest('trajectories', 'list', provenance='Histology track', probe_insertion=row['pid'])[0]
    # Note: Some can be Nan, take Planned instead
    if np.isnan(traj['x']):
        traj = one.alyx.rest('trajectories', 'list', provenance='Planned', probe_insertion=row['pid'])[0]
    
    # Create insertion from traj to get entry/tip points
    ins = Insertion(x=traj['x']/1e6, y=traj['y']/1e6, z=traj['z']/1e6, phi=traj['phi'], theta=traj['theta'], depth=traj['depth'], label='', beta=0)
    # Convert ins.entry, ins.tip into CCF coordinate framework
    bregma_mlapdv = np.array([ins.entry, ins.tip]) / 1e6 
    ccf_mlapdv = ba.xyz2ccf(bregma_mlapdv, ccf_order='mlapdv')
    # Save values in dataframe
    bwm.at[i, 'phi_e'] = traj['phi']  # TODO this is not in CCF
    bwm.at[i, 'theta_e'] = traj['theta']  # TODO this is not in CCF
    # Entry
    bwm.at[i, 'ml_e_e'] = ccf_mlapdv[0][0]
    bwm.at[i, 'ap_e_e'] = ccf_mlapdv[0][1]
    bwm.at[i, 'dv_e_e'] = ccf_mlapdv[0][2]
    # Tip
    bwm.at[i, 'ml_t_e'] = ccf_mlapdv[0][0]
    bwm.at[i, 'ap_t_e'] = ccf_mlapdv[0][1]
    bwm.at[i, 'dv_t_e'] = ccf_mlapdv[0][2]

In [27]:
probeNames = []
probePos = {}
probeAngle = {}
probeColor = {}
probeMaterial = {}
probeDepth = {}

for i,row in bwm.iterrows():
    name = 'p'+str(i)
    probeNames.append(name)
    
    probePos[name] = [row['ml_t_e'],row['ap_t_e'],row['dv_t_e']]
    probeAngle[name] = [row['phi_e'],row['theta_e'],0]
    probeColor[name] = rgb_to_hex((180,180,180))
    
    depth = vec_dist((row['ml_t_e'],row['ap_t_e'],row['dv_t_e']),(row['ml_e_e'],row['ap_e_e'],row['dv_e_e'])) 
    probeDepth[name] = [0.070, int(np.round(depth)/1000), 0.020]
    
    probeMaterial[name] = 'opaque-unlit'

In [28]:
urchin.setup()

(URN) connected to server
Login sent with ID: d82d8814, copy this ID into the renderer to connect.


In [29]:
# urchin.clear()

UnboundLocalError: local variable 'lines' referenced before assignment

In [30]:
areas = [22, 31, 44, 95, 184, 247, 254, 343, 453, 500, 512, 541, 623, 669, 677, 698, 703, 714, 895, 922, 972, 1057, 1089]

area_vis = {}
area_mat = {}
area_alpha = {}

for area in areas:
  area_vis[area] = True
  area_mat[area] = 'transparent-unlit'
  area_alpha[area] = 0.15

In [31]:
urchin.ccf.set_visibility(area_vis)
urchin.ccf.set_material(area_mat)
urchin.ccf.set_alpha(area_alpha)

urchin.ccf.set_color({'512':'#6D712E'})

AttributeError: module 'oursin' has no attribute 'ccf'

In [51]:
urchin.clear_probes()

In [14]:
urchin.probes.create(probeNames)
urchin.probes.set_position(probePos)
urchin.probes.set_angle(probeAngle)
urchin.probes.set_color(probeColor)
urchin.probes.set_size(probeDepth)

In [15]:
urchin.camera.set_rotation(45,0,135)